<a href="https://colab.research.google.com/github/ETSlime/GoogleColab/blob/Advance-Vision/Resnet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi -L

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
GPU 0: Tesla K80 (UUID: GPU-a9c4a19d-985d-7150-f26e-a374db91c613)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd drive/MyDrive

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-34-45c6c61b1a17>", line 1, in <module>
    get_ipython().magic('cd drive/MyDrive')
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 2160, in magic
    return self.run_line_magic(magic_name, magic_arg_s)
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 2081, in run_line_magic
    result = fn(*args,**kwargs)
  File "<decorator-gen-84>", line 2, in cd
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/magic.py", line 188, in <lambda>
    call = lambda f, *a, **k: f(*a, **k)
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/magics/osm.py", line 288, in cd
    oldcwd = py3compat.getcwd()
FileNotFoundError: [Errno 2] No such file or directory

During handling of the above exception, an

FileNotFoundError: ignored

In [ ]:
pwd

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from PIL import Image
from collections import OrderedDict
from tqdm import tqdm

# Import necessary PyTorch libraries
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision.models import resnet50
from torchvision import datasets, transforms, models

In [ ]:
with open("test.txt", "r") as f:
    test_label = f.read().splitlines()
    test_label = [elem[24:] for elem in test_label]
    test_label = list(map(int, test_label))

In [ ]:
import shutil
# generate validate set
if not os.path.exists('ImageNet/validate'):
    os.mkdir('ImageNet/validate')
rootdir = 'ImageNet/train'
list=os.listdir(rootdir)
for cls in list:
    path = os.path.join(rootdir, cls)
    des_path = os.path.join('ImageNet/validate', cls)
    if not os.path.exists(des_path):
        os.mkdir(des_path)
    for idx, item in enumerate(os.listdir(path)):
        full_path = os.path.join(path, item)
        if len(os.listdir(des_path)) < 5:
            shutil.move(full_path,des_path)

In [ ]:
class CustomDataset(torch.utils.data.Dataset):

    def __init__(self, img_path, labels, transform=None):
        self.path = img_path
        self.data_paths = [f for f in os.listdir(img_path)]
        self.labels = labels
        self.transform = transform

    def __getitem__(self, idx):
        img = Image.open(os.path.join(self.path,self.data_paths[idx]))
        img = img.convert("RGB")
        label = self.labels[idx]
        if self.transform:
            img = self.transform(img)
        
        return img, label

    def __len__(self):
        return len(self.data_paths)

In [ ]:
############## TODO: Data Augumentation ###########################
train_transforms = transforms.Compose([transforms.RandomHorizontalFlip(),
                                       transforms.RandomVerticalFlip(),
                                       transforms.RandomRotation(45),
                                       transforms.RandomAffine(45),
                                       transforms.ColorJitter(),
                                       transforms.ToTensor(),
                                       transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                            std=[0.229, 0.224, 0.225])])

####################################################################

test_transforms = transforms.Compose([transforms.Resize(256),
                                      transforms.TenCrop(224),
                                      transforms.Lambda(lambda crops: torch.stack([transforms.ToTensor()(crop) for crop in crops])),
                                      transforms.Lambda(lambda crops: torch.stack([transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])(crop) for crop in crops]))])
test_transforms_no_crop = transforms.Compose([transforms.Resize((224, 224)),
                                transforms.RandomHorizontalFlip(), 
                                transforms.ToTensor(),
                                transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])


In [ ]:
#for ablation study
no_crop = True
no_dropout = False

In [ ]:
train_dir = 'ImageNet/train'
train_data = datasets.ImageFolder(train_dir, train_transforms)

val_dir = 'ImageNet/validate'
validate_data = datasets.ImageFolder(val_dir, test_transforms)

test_dir = 'ImageNet/test'
if no_crop:
    test_data = CustomDataset(test_dir,test_label,test_transforms_no_crop)
else:
    test_data = CustomDataset(test_dir,test_label,test_transforms)

In [ ]:
validate,test = torch.utils.data.random_split(test_data, [5000, 45000])

In [ ]:
batch_size = 16
dataloaders = {}
dataloaders['train'] = DataLoader(train_data, batch_size=batch_size, shuffle=True)
dataloaders['validate'] = DataLoader(validate, batch_size=batch_size, shuffle=False)
dataloaders['test'] = DataLoader(test, batch_size=batch_size, shuffle=False)

In [ ]:
load_baseline = False
continue_train = True

In [ ]:
model = resnet50(pretrained=False)
if load_baseline:
    checkpoint = torch.load('ImageNet/resnet50_fconv_model_best.pth.tar')
    print('Load from baseline')
    new_state_dict = OrderedDict()
    for k, v in checkpoint['state_dict'].items():
        name = k[7:] # remove module.
        new_state_dict[name] = v

    model.load_state_dict(new_state_dict)
    if not no_dropout:
        print('dropout')
        model.fc = nn.Sequential(nn.BatchNorm1d(2048), nn.Dropout(0.2), nn.Linear(2048, 1000))

In [ ]:
sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
ct = 0
for child in model.children():
  ct += 1
  if ct < 10:
      for param in child.parameters():
          param.requires_grad = False

In [ ]:
sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
if continue_train:
    train_on_gpu = torch.cuda.is_available()
    print('Load from checkpoint')
    checkpoint = torch.load('ImageNet/res50_model_checkpoint.pth', map_location=torch.device('cuda'))
    model = checkpoint['model']
    criterion = checkpoint['criterion']
    optimizer = optim.SGD(model.parameters(), lr=0.01)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience = 5)
    n_epochs = 20
    valid_loss_min = checkpoint['valid_loss_min'] # track change in validation loss
else:
    # specify loss function (categorical cross-entropy)
    criterion = nn.CrossEntropyLoss()

    # specify optimizer (stochastic gradient descent with momentum)
    optimizer = optim.SGD(model.parameters(), lr=0.01)

    # secify learning rate scheduler (if there is no further decrease in loss for next 5 epochs 
    # then lower the learning rate by 0.1)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience = 5)

    train_on_gpu = torch.cuda.is_available()
    # Move the model to GPU if CUDA is available
    if train_on_gpu:
        model.cuda()

    # number of epochs to train the model
    n_epochs = 20

    valid_loss_min = np.Inf # track change in validation loss

In [ ]:
train_error = {}
train_acc = {}
validate_error = {}
validate_acc = {}

In [ ]:
for epoch in range(cur_epochs, n_epochs+1):
    # keep track of training and validation loss
    
    train_loss = 0.0
    train_accuracy = 0.0
    valid_loss = 0.0
    val_accuracy = 0.0
    

    ###################
    # train the model #
    ###################
    loop = tqdm(enumerate(dataloaders['train']),total=len(dataloaders['train']))
    model.train()
    for batch_idx, (data, target) in loop:
        
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update training loss
        train_loss += loss.item()*data.size(0)
        # Calculate training accuracy
        top_p, top_class = output.topk(1, dim=1)
        equals = top_class == target.view(*top_class.shape)
        train_accuracy += torch.mean(equals.type(torch.FloatTensor)).item()*data.size(0)
        
        loop.set_description(f'train Epoch[{epoch}/{n_epochs}]')
        loop.set_postfix(loss=train_loss/len(dataloaders['train'].dataset), acc=train_accuracy/((batch_idx+1)*batch_size))
        
        
        
    ######################    
    # validate the model #
    ######################
    loop = tqdm(enumerate(dataloaders['validate']),total=len(dataloaders['validate']),leave=False)
    model.eval()
    with torch.no_grad():
        for batch_idx, (data, target) in loop:
            # move tensors to GPU if CUDA is available
            if train_on_gpu:
                data, target = data.cuda(), target.cuda()

            input_var = torch.autograd.Variable(data, volatile=True)
            target_var = torch.autograd.Variable(target, volatile=True)
            bs, ncrops, c, h, w = input_var.size()
            # forward pass: compute predicted outputs by passing inputs to the model
            temp_output = model(input_var.view(-1, c, h, w))
            output = temp_output.view(bs, ncrops, -1).mean(1)
            # calculate the batch loss
            loss = criterion(output, target_var)
            # update average validation loss 
            valid_loss += loss.item()*data.size(0)
            # Calculate validation accuracy
            top_p, top_class = output.topk(1, dim=1)
            equals = top_class == target.view(*top_class.shape)
            val_accuracy += torch.mean(equals.type(torch.FloatTensor)).item()*data.size(0)
            
            loop.set_description(f'validate Epoch[{epoch}/{n_epochs}]')
            loop.set_postfix(loss=valid_loss/len(dataloaders['validate'].dataset), acc=val_accuracy/((batch_idx+1)*batch_size))
            
    
    # calculate average losses and accuracies
    train_loss = train_loss/len(dataloaders['train'].dataset)
    train_accuracy = train_accuracy/len(dataloaders['train'].dataset)
    valid_loss = valid_loss/len(dataloaders['validate'].dataset)
    val_accuracy = val_accuracy/len(dataloaders['validate'].dataset)
    
    train_error[epoch] = train_loss
    train_acc[epoch] = train_accuracy
    validate_error[epoch] = valid_loss
    validate_acc[epoch] = val_accuracy
    
    # Learning rate scheduler step
    scheduler.step(valid_loss) 
    
    # print training/validation statistics 
    print('Epoch: {} \tTraining Loss: {:.4f} \tTraining Accuracy: {:.4f} \tValidation Loss: {:.4f} \tValidation Accuracy: {:.4f}'.format(
        epoch, train_loss, train_accuracy, valid_loss, val_accuracy))
    
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min, valid_loss))
        
        checkpoint = {"model": model,
                      "criterion": criterion,
                      "epochs": epoch,
                      "optimizer_state": optimizer.state_dict(),
                      "model_state": model.state_dict(),
                      "valid_loss_min": valid_loss}
    
        # save model in colab
        torch.save(checkpoint, 'res50_model_checkpoint.pth')
        valid_loss_min = valid_loss
cur_epochs = n_epochs + 1
n_epochs = n_epochs + 20

In [ ]:
test_loss = 0.0
test_accuracy = 0.0
loop = tqdm(enumerate(dataloaders['test']),total=len(dataloaders['test']),leave=False)
model.eval()
top_class_overall = []
equals_overall = []
targets = []
with torch.no_grad():
    for batch_idx, (data, target) in loop:
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()

        input_var = torch.autograd.Variable(data, volatile=True)
        target_var = torch.autograd.Variable(target, volatile=True)
        bs, ncrops, c, h, w = input_var.size()
        # forward pass: compute predicted outputs by passing inputs to the model
        temp_output = model(input_var.view(-1, c, h, w))
        output = temp_output.view(bs, ncrops, -1).mean(1)
        # calculate the batch loss
        loss = criterion(output, target_var)
        # update average validation loss 
        test_loss += loss.item()*data.size(0)
        # Calculate validation accuracy
        top_p, top_class = output.topk(1, dim=1)
        equals = top_class == target.view(*top_class.shape)
        test_accuracy += torch.mean(equals.type(torch.FloatTensor)).item()*data.size(0)

        targets += target.cpu().numpy().tolist()
        top_class_overall += top_class.cpu().numpy().tolist()
        equals_overall += equals.cpu().numpy().tolist()

        loop.set_description(f'test Epoch[{epoch}/{n_epochs}]')
        loop.set_postfix(loss=test_loss/len(dataloaders['test'].dataset), acc=test_accuracy/((batch_idx+1)*batch_size))
test_loss = test_loss/len(dataloaders['test'].dataset)
test_accuracy = test_accuracy/len(dataloaders['test'].dataset)

In [ ]:
np.save('train_error', train_error)
np.save('train_acc', train_acc)
np.save('validate_error', validate_error)
np.save('validate_acc', validate_acc)

In [ ]:
train_error = np.load('save/train_error.npy', allow_pickle=True).item()
train_acc = np.load('save/train_acc.npy', allow_pickle=True).item()
validate_error = np.load('save/validate_error.npy', allow_pickle=True).item()
validate_acc = np.load('save/validate_acc.npy', allow_pickle=True).item()
test_acc = {}
test_loss = {}
test_acc[20] = np.load('save/test_accuracy_20.npy', allow_pickle=True).item()
test_acc[40] = np.load('save/test_accuracy_40.npy', allow_pickle=True).item()
test_acc[60] = np.load('save/test_accuracy_60.npy', allow_pickle=True).item()
test_acc[80] = np.load('save/test_accuracy_80.npy', allow_pickle=True).item()
test_loss[20] = np.load('save/test_loss_20.npy', allow_pickle=True).item()
test_loss[40] = np.load('save/test_loss_40.npy', allow_pickle=True).item()
test_loss[60] = np.load('save/test_loss_60.npy', allow_pickle=True).item()
test_loss[80] = np.load('save/test_loss_80.npy', allow_pickle=True).item()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
with PdfPages('train.pdf') as pdf:
    fig_1 = plt.figure(figsize=(8, 8))
    ax_1 = fig_1.add_subplot(211)
    ax_1.plot(train_error.keys(), train_error.values(), label='train error',linestyle='--')
    ax_1.plot(validate_error.keys(), validate_error.values(), label='validate error',linestyle='--')
    ax_1.legend(loc=0)
    ax_1.set_xlabel('Epoch number')
    ax_1.set_ylabel('Error')
    ax_1.set_title('Train and validate error and acc')
    ax_2 = fig_1.add_subplot(212)
    ax_2.plot(train_acc.keys(), train_acc.values(), label='train accuracy',linestyle='--')
    ax_2.plot(validate_acc.keys(), validate_acc.values(), label='validate accuracy',linestyle='--')
    ax_2.legend(loc=0)
    ax_2.set_xlabel('Epoch number')
    ax_2.set_ylabel('Accuracy')
    pdf.savefig(fig_1)
with PdfPages('test.pdf') as pdf:
    fig_2 = plt.figure(figsize=(8, 8))
    ax_3 = fig_2.add_subplot(211)
    ax_3.scatter(test_loss.keys(), test_loss.values(), label='test error')
    ax_3.plot(test_loss.keys(), test_loss.values(), linestyle='--')
    ax_3.legend(loc=0)
    ax_3.set_xlabel('Epoch number')
    ax_3.set_ylabel('Error')
    ax_3.set_title('Test error and acc')
    ax_4 = fig_2.add_subplot(212)
    ax_4.scatter(test_acc.keys(), test_acc.values(), label='test accuracy')
    ax_4.plot(test_acc.keys(), test_acc.values(), linestyle='--')
    ax_4.legend(loc=0)
    ax_4.set_xlabel('Epoch number')
    ax_4.set_ylabel('Accuracy')
    pdf.savefig(fig_2)

In [ ]:
#ablation study
def test(no_crop, dataloaders, model):
    loss = 0.0
    accuracy = 0.0
    loop = tqdm(enumerate(dataloaders['test']),total=len(dataloaders['test']),leave=False)
    model.eval()
    with torch.no_grad():
        for batch_idx, (data, target) in loop:
            # move tensors to GPU if CUDA is available
            if train_on_gpu:
                data, target = data.cuda(), target.cuda()
            if not no_crop:
                input_var = torch.autograd.Variable(data, volatile=True)
                target_var = torch.autograd.Variable(target, volatile=True)
                bs, ncrops, c, h, w = input_var.size()
                # forward pass: compute predicted outputs by passing inputs to the model
                temp_output = model(input_var.view(-1, c, h, w))
                output = temp_output.view(bs, ncrops, -1).mean(1)
                # calculate the batch loss
                loss = criterion(output, target_var)
            else:
                output = model(data)
                # calculate the batch loss
                loss = criterion(output, target)
                
            # update average validation loss 
            loss += loss.item()*data.size(0)
            # Calculate validation accuracy
            top_p, top_class = output.topk(1, dim=1)
            equals = top_class == target.view(*top_class.shape)
            accuracy += torch.mean(equals.type(torch.FloatTensor)).item()*data.size(0)
            loop.set_description(f'test Epoch[{epoch}/{n_epochs}]')
            loop.set_postfix(loss=loss/len(dataloaders['test'].dataset), acc=accuracy/((batch_idx+1)*batch_size))


    loss = loss/len(dataloaders['test'].dataset)
    accuracy = accuracy/len(dataloaders['test'].dataset)
    return loss, accuracy

In [ ]:
#no dropout
test_loss_no_dropout, test_accuracy_no_dropout = test(no_crop, dataloaders, model)
np.save('test_loss_no_dropout',test_loss_no_dropout)
np.save('test_accuracy_no_dropout',test_accuracy_no_dropout)
print('test_loss_no_dropout:',test_loss_no_dropout)
print('test_accuracy_no_dropout:',test_accuracy_no_dropout)

In [ ]:
#no crop
test_loss_no_crop, test_accuracy_no_crop = test(no_crop, dataloaders, model)
np.save('test_loss_no_crop',test_loss_no_crop)
np.save('test_accuracy_no_crop',test_accuracy_no_crop)
print('test_loss_no_crop:',test_loss_no_crop)
print('test_accuracy_no_crop:',test_accuracy_no_crop)

In [ ]:
#no crop no dropout
test_loss_no_crop_no_dropout, test_accuracy_no_crop_no_dropout = test(no_crop, dataloaders, model)
np.save('test_loss_no_crop_no_dropout',test_loss_no_crop_no_dropout)
np.save('test_accuracy_no_crop_no_dropout',test_accuracy_no_crop_no_dropout)
print('test_loss_no_crop_no_dropout:',test_loss_no_crop_no_dropout)
print('test_accuracy_no_crop_no_dropout:',test_accuracy_no_crop_no_dropout)